# LLM Классификация Отзывов: Полный Пайплайн (Промпт + BERT)

Цель: Классифицировать test.csv по категориям из categories.txt.  
Метрика: Weighted F1 (>0.90 цель).  
Ограничения: Только train для обучения/валидации; <5 сек/пример; Colab T4.

Этапы:
1. Установка.
2. Загрузка данных.
3. Разметка train (Mistral zero-shot).
4. Аугментация few-shot.
5. Промпт-инжиниринг (few-shot CoT).
6. Fine-tune BERT (Google multilingual) — опционально для +F1.
7. Инференс на test + submission.csv.

In [1]:
!pip install transformers accelerate bitsandbytes datasets pandas scikit-learn

In [2]:
!pip install torch torchvision

In [3]:
!pip install peft evaluate

In [4]:
import torch
import pandas as pd
import numpy as np
import time
import re
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import evaluate
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    AutoModelForSequenceClassification, TrainingArguments, Trainer,
    DataCollatorWithPadding, pipeline
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

In [5]:
print(f"GPU: {torch.cuda.is_available()}. VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

GPU: True. VRAM: 15.8 GB


## Разметка Train (Qwen Zero-Shot для Few-Shot)
Валидация только на holdout из train (test.csv не трогаем!).

In [6]:
# Категории
with open('categories.txt', 'r', encoding='utf-8') as f:
    categories = [line.strip() for line in f.readlines()]
categories_str = ", ".join(categories)
print("Категории:", categories)

# Train и Test
train_df = pd.read_csv('train.csv', names=['text'])  # Убираем кавычки и переносы
train_df['text'] = train_df['text'].str.strip().str.replace(r'^"', '', regex=True).str.replace(r'"$', '', regex=True)

test_df = pd.read_csv('test.csv', names=['text'])
test_df['text'] = test_df['text'].str.strip().str.replace(r'^"', '', regex=True).str.replace(r'"$', '', regex=True)

print(f"Train: {len(train_df)} отзывов\nTest: {len(test_df)} отзывов")

Категории: ['бытовая техника', 'обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей', 'украшения и аксессуары', 'электроника', 'нет товара']
Train: 1819 отзывов
Test: 7277 отзывов


In [7]:
model_name_qwen = "Qwen/Qwen2-7B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer_qwen = AutoTokenizer.from_pretrained(model_name_qwen, padding_side='left')
tokenizer_qwen.pad_token = tokenizer_qwen.eos_token

model_qwen = AutoModelForCausalLM.from_pretrained(
    model_name_qwen,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

def generate_response_qwen_batch(prompts, max_new_tokens=50, temperature=0.1):
    inputs = tokenizer_qwen(prompts, return_tensors="pt", truncation=True, padding=True, max_length=512).to("cuda")
    with torch.no_grad():
        outputs = model_qwen.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer_qwen.eos_token_id,
            eos_token_id=tokenizer_qwen.eos_token_id
        )
    responses = [tokenizer_qwen.decode(out[inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip() for out in outputs]
    return responses

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
def create_prompt(review_text):
    return f"""Ты эксперт по классификации отзывов на маркетплейсе. Категории: {categories_str}.

Правила:
- Выбери ОДНУ категорию. Если товар не упомянут или неясно — "нет товара".
- Ключевые слова: одежда (блузка, юбка, ткань, швы); обувь (ботинки, кроссовки); бытовая техника (стиралка, холодильник); посуда (тарелки, кастрюли); текстиль (постельное, шарфы); товары для детей (игрушки, коляска); украшения и аксессуары (кольца, сумки); электроника (телефон, наушники).
- Игнорируй доставку/эмоции, фокус на товаре.
- Ответ: ТОЛЬКО название категории.

Отзыв: {review_text}
Категория:"""

In [9]:
train_subset = train_df.sample(frac=0.2, random_state=42)  # ~2000 примеров
train_labels = []
start_time = time.time()
batch_size = 32

for i in range(0, len(train_subset), batch_size):
    batch = train_subset['text'][i:i+batch_size].tolist()
    prompts = [create_prompt(text) for text in batch]
    batch_responses = generate_response_qwen_batch(prompts)
    batch_labels = [re.search(r'(бытовая техника|обувь|одежда|посуда|текстиль|товары для детей|украшения и аксессуары|электроника|нет товара)', resp).group(0) if re.search(r'(бытовая техника|обувь|одежда|посуда|текстиль|товары для детей|украшения и аксессуары|электроника|нет товара)', resp) else "нет товара" for resp in batch_responses]
    train_labels.extend(batch_labels)
    print(f"Разметили {i+len(batch)}/{len(train_subset)}")

avg_time_label = (time.time() - start_time) / len(train_subset)
print(f"Среднее время разметки train (подвыборка): {avg_time_label:.2f} сек")

train_subset['label'] = train_labels
train_df = train_subset.copy()  # Для совместимости с остальным кодом
train_df.to_csv('train_labeled.csv', index=False)

# Валидация на holdout (10% подвыборки)
holdout = train_df.sample(frac=0.1, random_state=42)
print("Разметка готова")

Разметили 32/364
Разметили 64/364
Разметили 96/364
Разметили 128/364
Разметили 160/364
Разметили 192/364
Разметили 224/364
Разметили 256/364
Разметили 288/364
Разметили 320/364
Разметили 352/364
Разметили 364/364
Среднее время разметки train (подвыборка): 3.03 сек
Разметка готова


## Аугментация Few-Shot Примеров
Выберем 12+ примеров (баланс) и аугментируем парафразами.

In [12]:
def generate_response_qwen_single(prompt, max_new_tokens=100, temperature=0.1):
    inputs = tokenizer_qwen([prompt], return_tensors="pt", truncation=True, padding=True, max_length=512).to("cuda")
    with torch.no_grad():
        outputs = model_qwen.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer_qwen.eos_token_id,
            eos_token_id=tokenizer_qwen.eos_token_id
        )
    response = tokenizer_qwen.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()
    return response

In [13]:
few_shot_df = train_df.groupby('label').apply(lambda x: x.sample(min(2, len(x)), random_state=42)).reset_index(drop=True)
few_shot_examples = few_shot_df.to_dict('records')[:12]  # Топ-12

# Парафразинг
def paraphrase_example(example_text):
    para_prompt = f"Перефразируй этот отзыв, сохраняя смысл и ключевые слова: {example_text}"
    para_text = generate_response_qwen_single(para_prompt, max_new_tokens=100)
    return para_text

augmented_examples = []
for ex in few_shot_examples:
    augmented_examples.append({'text': ex['text'], 'label': ex['label']})
    for _ in range(2):  # 2 парафраза
        para_text = paraphrase_example(ex['text'])
        augmented_examples.append({'text': para_text, 'label': ex['label']})

few_shot_str = "\n".join([f"Отзыв: {ex['text']}\nКатегория: {ex['label']}\n" for ex in augmented_examples[:12]])
print("Few-shot готов. Примеров:", len(augmented_examples))

/tmp/ipython-input-2031110604.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  few_shot_df = train_df.groupby('label').apply(lambda x: x.sample(min(2, len(x)), random_state=42)).reset_index(drop=True)


Few-shot готов. Примеров: 36


## Промпт-Инжиниринг (Few-Shot CoT для Test)

In [ ]:
def create_prompt(review_text):
    return f"""Ты эксперт по классификации отзывов на маркетплейсе. Категории: {categories_str}.

Правила:
- Выбери ОДНУ категорию. Если товар не упомянут или неясно — "нет товара".
- Ключевые слова: одежда (блузка, юбка, ткань, швы); обувь (ботинки, кроссовки); бытовая техника (стиралка, холодильник); посуда (тарелки, кастрюли); текстиль (постельное, шарфы); товары для детей (игрушки, коляска); украшения и аксессуары (кольца, сумки); электроника (телефон, наушники).
- Игнорируй доставку/эмоции, фокус на товаре.
- Ответ: ТОЛЬКО название категории.

Отзыв: {review_text}
Категория:"""

test_prompt_labels = []
start_time = time.time()
batch_size = 64  # Увеличено для скорости, проверяй VRAM

torch.cuda.empty_cache()  # Очистка VRAM перед началом

for i in range(0, len(test_df), batch_size):
    batch = test_df['text'][i:i+batch_size].tolist()
    prompts = [create_prompt(text) for text in batch]
    batch_responses = generate_response_qwen_batch(prompts, max_new_tokens=10, temperature=0.1)  # Из ячейки 6
    batch_labels = [
        re.search(r'(бытовая техника|обувь|одежда|посуда|текстиль|товары для детей|украшения и аксессуары|электроника|нет товара)', resp).group(0)
        if re.search(r'(бытовая техника|обувь|одежда|посуда|текстиль|товары для детей|украшения и аксессуары|электроника|нет товара)', resp)
        else "нет товара"
        for resp in batch_responses
    ]
    test_prompt_labels.extend(batch_labels)
    print(f"Классифицировали {i+len(batch)}/{len(test_df)}")
    torch.cuda.empty_cache()  # Очистка VRAM после батча

avg_time_prompt = (time.time() - start_time) / len(test_df)
print(f"Среднее время промпт на test: {avg_time_prompt:.2f} сек")
test_df['predicted_category_prompt'] = test_prompt_labels

# Сохранение промежуточных результатов (на случай сбоя)
test_df[['text', 'predicted_category_prompt']].to_csv('test_prompt_temp.csv', index=False)
print("Промежуточные результаты сохранены в test_prompt_temp.csv")

Классифицировали 64/7277
Классифицировали 128/7277
Классифицировали 192/7277
Классифицировали 256/7277
Классифицировали 320/7277


## Fine-Tune BERT (Google Multilingual) — Опционально для Топ-F1
Заменит промпт на supervised модель. Валидация только на holdout из train.

In [ ]:
# Подготовка данных для BERT (только train)
le = LabelEncoder()
train_df['label_id'] = le.fit_transform(train_df['label'])
num_labels = len(le.classes_)

train_split, val_split = train_test_split(train_df, test_size=0.1, stratify=train_df['label_id'], random_state=42)
train_dataset = Dataset.from_pandas(train_split[['text', 'label_id']])
val_dataset = Dataset.from_pandas(val_split[['text', 'label_id']])

# Аугментация: Шум в train
def augment_text(text):
    words = text.split()
    if len(words) > 3 and np.random.rand() > 0.5:
        np.random.shuffle(words)
        return ' '.join(words)
    return text

train_dataset = train_dataset.map(lambda x: {'text': augment_text(x['text'])})
model_name_bert = "google-bert/bert-base-multilingual-cased"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
if tokenizer_bert.pad_token is None:
    tokenizer_bert.pad_token = tokenizer_bert.unk_token

def tokenize_function(examples):
    return tokenizer_bert(examples['text'], truncation=True, padding=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label_id'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label_id'])

In [ ]:
# Модель BERT + LoRA
model_bert = AutoModelForSequenceClassification.from_pretrained(
    model_name_bert,
    num_labels=num_labels,
    id2label={i: label for i, label in enumerate(le.classes_)},
    label2id={label: i for i, label in enumerate(le.classes_)}
)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "value"]
)
model_bert = get_peft_model(model_bert, lora_config)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer_bert)

metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [ ]:
# Training
training_args = TrainingArguments(
    output_dir="./bert-finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    report_to="none"
)

trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer_bert,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
val_results = trainer.evaluate()
print(f"Валидационный Weighted F1 (holdout из train): {val_results['eval_f1']:.4f}")

trainer.save_model("./bert-final")
tokenizer_bert.save_pretrained("./bert-final")

## Инференс на Test + Submission
'prompt' или 'bert' (по умолчанию BERT, если обучен).

In [ ]:
method = 'bert'  # Или 'prompt'

if method == 'bert':
    classifier = pipeline("text-classification", model="./bert-final", tokenizer="./bert-final", device=0)
    def classify_bert(text):
        result = classifier(text)
        pred_id = int(result[0]['label'].split('_')[-1]) if '_' in result[0]['label'] else int(result[0]['label'])
        return le.classes_[pred_id]

    start_time = time.time()
    test_df['predicted_category'] = test_df['text'].apply(classify_bert)
    avg_time_bert = (time.time() - start_time) / len(test_df)
    print(f"Среднее время BERT на test: {avg_time_bert:.2f} сек")
else:  # Промпт
    test_df['predicted_category'] = test_df['predicted_category_prompt']
    print(f"Используем промпт. Время: {avg_time_prompt:.2f} сек")

# Submission CSV (формат: category\npred1\n...)
test_df[['predicted_category']].to_csv('submission.csv', index=False, header=False)
print("\nsubmission.csv готов! Статистика:")
print(test_df['predicted_category'].value_counts())

# Пример валидации на holdout (если есть manual_labels)
# f1_final = f1_score(manual_labels, holdout['predicted_category'].tolist()[:len(manual_labels)], average='weighted')
# print(f"Примерный F1: {f1_final}")
print("Готово! Скачай submission.csv.")